# pandas的拼接操作

In [2]:
import pandas as pd
from pandas import DataFrame
import numpy as np


pandas的拼接分为两种：
- 级联：pd.concat, pd.append
- 合并：pd.merge, pd.join

## 0. 回顾numpy的级联

============================================

练习12：

1. 生成2个3*3的矩阵，对其分别进行两个维度上的级联

============================================

In [3]:
n1 = np.random.randint(0,150,size=(3,3))
n2 = np.random.randint(0,150,size=(3,3))
display(n1,n2)

array([[ 32,   5,  95],
       [ 61, 104, 103],
       [104,   9,  21]])

array([[ 59,  95,  79],
       [130,  75,  82],
       [  2,  89, 137]])

In [4]:
# 垂直级联
np.concatenate((n1,n2))

array([[ 32,   5,  95],
       [ 61, 104, 103],
       [104,   9,  21],
       [ 59,  95,  79],
       [130,  75,  82],
       [  2,  89, 137]])

In [5]:
# 水平
np.concatenate((n1,n2), axis=1)

array([[ 32,   5,  95,  59,  95,  79],
       [ 61, 104, 103, 130,  75,  82],
       [104,   9,  21,   2,  89, 137]])

In [8]:
# np.vstack((n1,n2))
np.hstack((n1,n2))

array([[ 32,   5,  95,  59,  95,  79],
       [ 61, 104, 103, 130,  75,  82],
       [104,   9,  21,   2,  89, 137]])

为方便讲解，我们首先定义一个生成DataFrame的函数：

In [9]:
def mk_df(index,cols):
    data = {col:[col + str(i) for i in index]    for col in cols}
    return DataFrame(data=data,index=index,columns=cols)

In [10]:
mk_df([1,2,3,4],list('abcd'))

,a,b,c,d
1,a1,b1,c1,d1
2,a2,b2,c2,d2
3,a3,b3,c3,d3
4,a4,b4,c4,d4


## 1. 使用pd.concat()级联

pandas使用pd.concat函数，与np.concatenate函数类似，只是多了一些参数：
```
objs
axis=0
join='outer'
join_axes=None
ignore_index=False
keys = [value1,value2...]
```

### 1)  简单级联

In [18]:
df1 = mk_df([1,2,3,4],list('abcd'))
df2 = mk_df([1,2,3,4],list('abcd'))
df1
df2

,a,b,c,d
1,a1,b1,c1,d1
2,a2,b2,c2,d2
3,a3,b3,c3,d3
4,a4,b4,c4,d4


In [26]:
pd.concat((df1,df2),ignore_index=True)

,a,b,c,d
0,a1,b1,c1,d1
1,a2,b2,c2,d2
2,a3,b3,c3,d3
3,a4,b4,c4,d4
4,a1,b1,c1,d1
5,a2,b2,c2,d2
6,a3,b3,c3,d3
7,a4,b4,c4,d4


和np.concatenate一样，优先增加行数（默认axis=0）

可以通过设置axis来改变级联方向

级连会把该方向上索引相同的元素放在一行（一列），index/columns在级联时可以重复

也可以选择忽略ignore_index，重新索引

或者使用多层索引 keys  

concat([x,y],keys=['x','y'])

In [28]:
pd.concat((df1,df2),keys=['df1','df2'])

a   b   c   d
df1 1  a1  b1  c1  d1
    2  a2  b2  c2  d2
    3  a3  b3  c3  d3
    4  a4  b4  c4  d4
df2 1  a1  b1  c1  d1
    2  a2  b2  c2  d2
    3  a3  b3  c3  d3
    4  a4  b4  c4  d4

============================================

练习13：

1. 想一想级联的应用场景？

2. 使用昨天的知识，建立一个期中考试张三、李四的成绩表ddd

3. 假设新增考试学科"计算机"，如何实现？

4. 新增王老五同学的成绩，如何实现？

============================================

In [33]:
columns = list('abcd')
index = ['zs','ls']
data = np.random.randint(0,150,size=(2,4))
df = DataFrame(data=data,index=index,columns=columns)
df

,a,b,c,d
zs,20,5,21,34
ls,139,106,87,20


In [40]:
index = ['zs','ls']
columns=['计算机']
data = np.random.randint(0,150,size=(2,1))
df1 = DataFrame(data=data,index=index,columns=columns)
df1

,计算机
zs,98
ls,142


In [55]:
df0 = pd.concat((df,df1),axis=1)
df0

,a,b,c,d,计算机
zs,20,5,21,34,98
ls,139,106,87,20,142


In [42]:
index = ['ww']
columns=['a','b','c','d','计算机']
data = np.random.randint(0,150,size=(1,5))
df2 = DataFrame(data=data,index=index,columns=columns)
df2

,a,b,c,d,计算机
ww,75,100,32,119,99


In [70]:
df3 = pd.concat((df0,df2),sort=True,join='inner')
df3

,a,b,c,d,计算机
zs,20,5,21,34,98
ls,139,106,87,20,142
ww,75,100,32,119,99


In [66]:
pd.concat((df0,df2),sort=True,axis=1,join_axes=[df1.index])

,a,b,c,d,计算机,a,b,c,d,计算机
zs,20,5,21,34,98,NaN,NaN,NaN,NaN,NaN
ls,139,106,87,20,142,NaN,NaN,NaN,NaN,NaN


### 2) 不匹配级联

不匹配指的是级联的维度的索引不一致。例如纵向级联时列索引不一致，横向级联时行索引不一致

有3种连接方式：

- 外连接：补NaN（默认模式）

- 内连接：只连接匹配的项

- 连接指定轴 join_axes

============================================

练习14：

    假设【期末】考试ddd2的成绩没有张三的，只有李四、王老五、赵小六的，使用多种方法级联

============================================

In [86]:
columns = list('abcd')
index = ['ls','aa','bb']
data = np.random.randint(0,150,size=(3,4))
dff = DataFrame(data=data,index=index,columns=columns)
dff

,a,b,c,d
ls,68,11,113,116
aa,30,102,40,109
bb,92,19,15,103


In [87]:
columns = list('abcd')
index = ['zs','ls','aa','bb']
data = np.random.randint(0,150,size=(4,4))
dfff = DataFrame(data=data,index=index,columns=columns)
dfff

,a,b,c,d
zs,108,25,3,51
ls,126,97,131,3
aa,72,28,2,23
bb,101,138,127,30


In [91]:
pd.concat((dff,dfff),keys=['1','2'],sort=True,axis=1)

1                         2              
       a      b      c      d    a    b    c   d
aa  30.0  102.0   40.0  109.0   72   28    2  23
bb  92.0   19.0   15.0  103.0  101  138  127  30
ls  68.0   11.0  113.0  116.0  126   97  131   3
zs   NaN    NaN    NaN    NaN  108   25    3  51

### 3) 使用append()函数添加

由于在后面级联的使用非常普遍，因此有一个函数append专门用于在后面添加

In [92]:
dff.append(dfff)

,a,b,c,d
ls,68,11,113,116
aa,30,102,40,109
bb,92,19,15,103
zs,108,25,3,51
ls,126,97,131,3
aa,72,28,2,23
bb,101,138,127,30


============================================

练习15：

    新建一个只有张三李四王老五的期末考试成绩单ddd3，使用append()与期中考试成绩表ddd级联

============================================

使用pd.read...读取文件

## 2. 使用pd.merge()合并

merge与concat的区别在于，merge需要依据某一共同的行或列来进行合并

使用pd.merge()合并时，会自动根据两者相同column名称的那一列，作为key来进行合并。

注意每一列元素的顺序不要求一致

###  1) 一对一合并

In [93]:
df1 = DataFrame({'name':['张三','李四','Chales'],'id':[1,2,3],'age':[22,21,25]})

df2 = DataFrame({'sex':['男','男','女'],'id':[2,3,4],'group':['sale','search','service']})
display(df1,df2)

,name,id,age
0,张三,1,22
1,李四,2,21
2,Chales,3,25


,sex,id,group
0,男,2,sale
1,男,3,search
2,女,4,service


In [94]:
pd.merge(df1,df2)

,name,id,age,sex,group
0,李四,2,21,男,sale
1,Chales,3,25,男,search


### 2) 多对一合并

In [95]:
df1 = DataFrame({'name':['张三','李四','Chales'],'id':[1,2,2],'age':[22,21,25]})

df2 = DataFrame({'sex':['男','男','女'],'id':[2,3,4],'group':['sale','search','service']})
display(df1,df2)


,name,id,age
0,张三,1,22
1,李四,2,21
2,Chales,2,25


,sex,id,group
0,男,2,sale
1,男,3,search
2,女,4,service


In [97]:
# pd.merge(df1,df2)
df1.merge(df2)

,name,id,age,sex,group
0,李四,2,21,男,sale
1,Chales,2,25,男,sale


### 3) 多对多合并

In [100]:
df1 = DataFrame({'name':['张三','李四','张三'],'salary':[10000,12000,20000],'age':[22,21,25]})

df2 = DataFrame({'sex':['男','男','女'],'name':['张三','张三','凡凡'],'group':['sale','search','service']})
display(df1,df2)

,name,salary,age
0,张三,10000,22
1,李四,12000,21
2,张三,20000,25


,sex,name,group
0,男,张三,sale
1,男,张三,search
2,女,凡凡,service


In [101]:
pd.merge(df1,df2)

,name,salary,age,sex,group
0,张三,10000,22,男,sale
1,张三,10000,22,男,search
2,张三,20000,25,男,sale
3,张三,20000,25,男,search


### 4) key的规范化

- 使用on=显式指定哪一列为key,当有多个key相同时使用

In [102]:
df1 = DataFrame({'name':['张三','李四','张三'],'salary':[10000,12000,20000],'age':[22,21,25]})

df2 = DataFrame({'age':[21,18,29],'name':['张三','张三','凡凡'],'group':['sale','search','service']})
display(df1,df2)

,name,salary,age
0,张三,10000,22
1,李四,12000,21
2,张三,20000,25


,age,name,group
0,21,张三,sale
1,18,张三,search
2,29,凡凡,service


In [104]:
pd.merge(df1,df2,on='name',suffixes=['_df1','_df2'])

,name,salary,age_df1,age_df2,group
0,张三,10000,22,21,sale
1,张三,10000,22,18,search
2,张三,20000,25,21,sale
3,张三,20000,25,18,search


- 使用left_on和right_on指定左右两边的列作为key，当左右两边的key都不相等时使用

In [105]:
df1 = DataFrame({'name':['张三','李四','张三'],'salary':[10000,12000,20000],'age':[22,21,25]})

df2 = DataFrame({'年龄':[21,18,29],'名字':['张三','张三','凡凡'],'group':['sale','search','service']})
display(df1,df2)

,name,salary,age
0,张三,10000,22
1,李四,12000,21
2,张三,20000,25


,年龄,名字,group
0,21,张三,sale
1,18,张三,search
2,29,凡凡,service


In [106]:
pd.merge(df1,df2,left_on='name',right_on='名字')

,name,salary,age,年龄,名字,group
0,张三,10000,22,21,张三,sale
1,张三,10000,22,18,张三,search
2,张三,20000,25,21,张三,sale
3,张三,20000,25,18,张三,search


当左边的列和右边的index相同的时候,使用right_index=True

In [107]:
df1 = DataFrame({'name':['张三','李四','张三'],'salary':[10000,12000,20000],'age':[22,21,25]})

df2 = DataFrame({'年龄':[21,18,29],'名字':['张三','张三','凡凡'],'group':['sale','search','service']},
                index = [22,21,25])
display(df1,df2)

,name,salary,age
0,张三,10000,22
1,李四,12000,21
2,张三,20000,25


,年龄,名字,group
22,21,张三,sale
21,18,张三,search
25,29,凡凡,service


In [108]:
pd.merge(df1,df2,left_on='age',right_index=True)

,name,salary,age,年龄,名字,group
0,张三,10000,22,21,张三,sale
1,李四,12000,21,18,张三,search
2,张三,20000,25,29,凡凡,service


============================================

练习16：

1. 假设有两份成绩单，除了ddd是张三李四王老五之外，还有ddd4是张三和赵小六的成绩单，如何合并？

2. 如果ddd4中张三的名字被打错了，成为了张十三，怎么办？

3. 自行练习多对一，多对多的情况  

4. 自学left_index,right_index

============================================

### 5) 内合并与外合并

- 内合并：只保留两者都有的key（默认模式）

In [109]:
df1 = DataFrame({'name':['张三','李四','张三'],'salary':[10000,12000,20000],'age':[22,21,25]})

df2 = DataFrame({'age':[21,18,29],'名字':['张三','张三','凡凡'],'group':['sale','search','service']})
display(df1,df2)

,name,salary,age
0,张三,10000,22
1,李四,12000,21
2,张三,20000,25


,age,名字,group
0,21,张三,sale
1,18,张三,search
2,29,凡凡,service


In [110]:
pd.merge(df1,df2,left_on='name',right_on='名字')

,name,salary,age_x,age_y,名字,group
0,张三,10000,22,21,张三,sale
1,张三,10000,22,18,张三,search
2,张三,20000,25,21,张三,sale
3,张三,20000,25,18,张三,search


- 外合并 how='outer'：补NaN

In [111]:
pd.merge(df1,df2,left_on='name',right_on='名字',how = 'outer')

,name,salary,age_x,age_y,名字,group
0,张三,10000.0,22.0,21.0,张三,sale
1,张三,10000.0,22.0,18.0,张三,search
2,张三,20000.0,25.0,21.0,张三,sale
3,张三,20000.0,25.0,18.0,张三,search
4,李四,12000.0,21.0,NaN,NaN,NaN
5,NaN,NaN,NaN,29.0,凡凡,service


- 左合并、右合并：how='left'，how='right'，

In [113]:
pd.merge(df1,df2,left_on='name',right_on='名字',how='right')

,name,salary,age_x,age_y,名字,group
0,张三,10000.0,22.0,21,张三,sale
1,张三,20000.0,25.0,21,张三,sale
2,张三,10000.0,22.0,18,张三,search
3,张三,20000.0,25.0,18,张三,search
4,NaN,NaN,NaN,29,凡凡,service


============================================

练习17：



1. 考虑应用情景，使用多种方式合并ddd与ddd4

============================================

### 6) 列冲突的解决

当列冲突时，即有多个列名称相同时，需要使用on=来指定哪一个列作为key，配合suffixes指定冲突列名

In [118]:
#期中
df1 = DataFrame({'name':['张三','李四','张三'],'degree':[120,118,149],'age':[22,21,25]})

#期末考试
df2 = DataFrame({'degree':[99,97,129],'name':['张三','张三','凡凡'],'group':['sale','search','service']})
display(df1,df2)
pd.merge(df1,df2,how='outer')

,name,degree,age
0,张三,120,22
1,李四,118,21
2,张三,149,25


,degree,name,group
0,99,张三,sale
1,97,张三,search
2,129,凡凡,service


,name,degree,age,group
0,张三,120,22.0,NaN
1,李四,118,21.0,NaN
2,张三,149,25.0,NaN
3,张三,99,NaN,sale
4,张三,97,NaN,search
5,凡凡,129,NaN,service


可以使用suffixes=自己指定后缀

============================================

练习18：

    假设有两个同学都叫李四，ddd5、ddd6都是张三和李四的成绩表，如何合并？

============================================

## 作业
## 3. 案例分析：美国各州人口数据分析

作业知识补充

In [130]:
# unique() 去重函数
import numpy as np
import pandas as pd
from pandas import Series
s = Series(['Tom','Lucy','Tom','dancer','Lucy'])

In [131]:
#使用pandas读取数据
pop = pd.read_csv('../data/state-population.csv')

areas = pd.read_csv('../data/state-areas.csv')

abb = pd.read_csv('../data/state-abbrevs.csv')

In [132]:
s.unique()

array(['Tom', 'Lucy', 'dancer'], dtype=object)

In [133]:
n = DataFrame({'name':['Tom','Lucy','Tom','dancer','Lucy'],'age':[12,13,12,11,15]})
n

,name,age
0,Tom,12
1,Lucy,13
2,Tom,12
3,dancer,11
4,Lucy,15


In [134]:
# query 条件查询函数
n.query("name == 'Lucy' & age>14")  

,name,age
4,Lucy,15


首先导入文件，并查看数据样本

In [135]:
s_abb = pd.read_csv('../data/state-abbrevs.csv')
s_abb.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [136]:
s_pop = pd.read_csv('../data/state-population.csv')
s_pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [137]:
s_ares = pd.read_csv('../data/state-areas.csv')
s_ares.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


合并pop与abbrevs两个DataFrame，分别依据state/region列和abbreviation列来合并。

为了保留所有信息，使用外合并。

In [140]:
merge = pd.merge(s_abb,s_pop,left_on='abbreviation',right_on='state/region',how='outer')
merge

,state,abbreviation,state/region,ages,year,population
0,Alabama,AL,AL,under18,2012,1117489.0
1,Alabama,AL,AL,total,2012,4817528.0
2,Alabama,AL,AL,under18,2010,1130966.0
3,Alabama,AL,AL,total,2010,4785570.0
4,Alabama,AL,AL,under18,2011,1125763.0
5,Alabama,AL,AL,total,2011,4801627.0
6,Alabama,AL,AL,total,2009,4757938.0
7,Alabama,AL,AL,under18,2009,1134192.0
8,Alabama,AL,AL,under18,2013,1111481.0
9,Alabama,AL,AL,total,2013,4833722.0


去除abbreviation的那一列（axis=1）

In [146]:
merge1 = merge.drop('abbreviation', axis=1)
merge1

,state,state/region,ages,year,population
0,Alabama,AL,under18,2012,1117489.0
1,Alabama,AL,total,2012,4817528.0
2,Alabama,AL,under18,2010,1130966.0
3,Alabama,AL,total,2010,4785570.0
4,Alabama,AL,under18,2011,1125763.0
5,Alabama,AL,total,2011,4801627.0
6,Alabama,AL,total,2009,4757938.0
7,Alabama,AL,under18,2009,1134192.0
8,Alabama,AL,under18,2013,1111481.0
9,Alabama,AL,total,2013,4833722.0


查看存在缺失数据的列。

使用.isnull().any()，只有某一列存在一个缺失数据，就会显示True。

In [147]:
merge1.isnull().any()

state            True
state/region    False
ages            False
year            False
population       True
dtype: bool

查看缺失数据

In [150]:
merge1.loc[merge1.isnull().any(axis=1)]

,state,state/region,ages,year,population
2448,NaN,PR,under18,1990,NaN
2449,NaN,PR,total,1990,NaN
2450,NaN,PR,total,1991,NaN
2451,NaN,PR,under18,1991,NaN
2452,NaN,PR,total,1993,NaN
2453,NaN,PR,under18,1993,NaN
2454,NaN,PR,under18,1992,NaN
2455,NaN,PR,total,1992,NaN
2456,NaN,PR,under18,1994,NaN
2457,NaN,PR,total,1994,NaN


根据数据是否缺失情况显示数据，如果缺失为True，那么显示

In [154]:
merge1.loc[merge1.isnull().any(ffill_values=True,axis=1)]

TypeError: any() got an unexpected keyword argument 'ffill_values'

找到有哪些state/region使得state的值为NaN，使用unique()查看非重复值

In [164]:
a = merge1['state'].isnull()
merge1['state/region'][a].unique()

array(['PR', 'USA'], dtype=object)

为找到的这些state/region的state项补上正确的值，从而去除掉state这一列的所有NaN！

记住这样清除缺失数据NaN的方法！

In [165]:
s_ares

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707
5,Colorado,104100
6,Connecticut,5544
7,Delaware,1954
8,Florida,65758
9,Georgia,59441


In [171]:
merge_area = merge1.merge(s_ares,how='outer')
merge_area

,state,state/region,ages,year,population,area (sq. mi)
0,Alabama,AL,under18,2012.0,1117489.0,52423.0
1,Alabama,AL,total,2012.0,4817528.0,52423.0
2,Alabama,AL,under18,2010.0,1130966.0,52423.0
3,Alabama,AL,total,2010.0,4785570.0,52423.0
4,Alabama,AL,under18,2011.0,1125763.0,52423.0
5,Alabama,AL,total,2011.0,4801627.0,52423.0
6,Alabama,AL,total,2009.0,4757938.0,52423.0
7,Alabama,AL,under18,2009.0,1134192.0,52423.0
8,Alabama,AL,under18,2013.0,1111481.0,52423.0
9,Alabama,AL,total,2013.0,4833722.0,52423.0


继续寻找存在缺失数据的列

In [174]:
merge_area.isnull().any()

state            True
state/region     True
ages             True
year             True
population       True
area (sq. mi)    True
dtype: bool

我们会发现area(sq.mi)这一列有缺失数据，为了找出是哪一行，我们需要找出是哪个state没有数据

In [176]:
kong = merge_area['area (sq. mi)'].isnull()
kong

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
2515     True
2516     True
2517     True
2518     True
2519     True
2520     True
2521     True
2522     True
2523     True
2524     True
2525     True
2526     True
2527     True
2528     True
2529     True
2530     True
2531     True
2532     True
2533     True
2534     True
2535     True
2536     True
2537     True
2538     True
2539     True
2540     True
2541     True
2542     True
2543     True
2544    False
Name: area (sq. mi), Length: 2545, dtype: bool

去除含有缺失数据的行

In [183]:
merge2 = merge_area.dropna()
merge2

,state,state/region,ages,year,population,area (sq. mi)
0,Alabama,AL,under18,2012.0,1117489.0,52423.0
1,Alabama,AL,total,2012.0,4817528.0,52423.0
2,Alabama,AL,under18,2010.0,1130966.0,52423.0
3,Alabama,AL,total,2010.0,4785570.0,52423.0
4,Alabama,AL,under18,2011.0,1125763.0,52423.0
5,Alabama,AL,total,2011.0,4801627.0,52423.0
6,Alabama,AL,total,2009.0,4757938.0,52423.0
7,Alabama,AL,under18,2009.0,1134192.0,52423.0
8,Alabama,AL,under18,2013.0,1111481.0,52423.0
9,Alabama,AL,total,2013.0,4833722.0,52423.0


查看数据是否缺失

In [182]:
merge_area.dropna().isnull().any()

state            False
state/region     False
ages             False
year             False
population       False
area (sq. mi)    False
dtype: bool

找出2010年的全民人口数据,df.query(查询语句)

In [187]:
m2010 = merge2.query("year==2010")
m2010

,state,state/region,ages,year,population,area (sq. mi)
2,Alabama,AL,under18,2010.0,1130966.0,52423.0
3,Alabama,AL,total,2010.0,4785570.0,52423.0
90,Alaska,AK,under18,2010.0,187902.0,656425.0
91,Alaska,AK,total,2010.0,713868.0,656425.0
100,Arizona,AZ,under18,2010.0,1628563.0,114006.0
101,Arizona,AZ,total,2010.0,6408790.0,114006.0
188,Arkansas,AR,under18,2010.0,711947.0,53182.0
189,Arkansas,AR,total,2010.0,2922280.0,53182.0
196,California,CA,under18,2010.0,9284094.0,163707.0
197,California,CA,total,2010.0,37333601.0,163707.0


对查询结果进行处理，以state列作为新的行索引:set_index

In [188]:
m2010.set_index('state')

,state/region,ages,year,population,area (sq. mi)
state,,,,,
Alabama,AL,under18,2010.0,1130966.0,52423.0
Alabama,AL,total,2010.0,4785570.0,52423.0
Alaska,AK,under18,2010.0,187902.0,656425.0
Alaska,AK,total,2010.0,713868.0,656425.0
Arizona,AZ,under18,2010.0,1628563.0,114006.0
Arizona,AZ,total,2010.0,6408790.0,114006.0
Arkansas,AR,under18,2010.0,711947.0,53182.0
Arkansas,AR,total,2010.0,2922280.0,53182.0
California,CA,under18,2010.0,9284094.0,163707.0


计算人口密度。注意是Series/Series，其结果还是一个Series。

In [189]:
p = m2010['population']/m2010['area (sq. mi)']
p

2         21.573851
3         91.287603
90         0.286251
91         1.087509
100       14.284889
101       56.214497
188       13.386992
189       54.948667
196       56.711649
197      228.051342
282       11.783084
283       48.493718
292      146.859127
293      645.600649
378      105.157625
379      460.445752
388     1489.838235
389     8898.897059
474       60.821984
475      286.597129
484       41.905150
485      163.409902
570      124.746707
571       27.791072
580        5.132709
581       18.794338
666      221.687472
667       53.905383
676       44.093438
677      178.197831
           ...     
1732      15.569868
1733      61.321530
1818      86.015622
1819      20.428381
1828      60.474098
1829     275.966651
1914     681.339159
1915     144.393528
1924      33.741931
1925     144.854594
2010      10.583512
2011       2.634107
2020      35.474066
2021     150.825298
2106      93.987655
2107      25.610530
2116      10.282425
2117      32.677188
2202      65.085075


ValueError: can not merge DataFrame with instance of type <class 'pandas.core.series.Series'>

排序，并找出人口密度最高的五个州sort_values()

In [193]:
type(p)

pandas.core.series.Series

In [199]:
p.sort_values(inplace=True)

p.sort_values()[-5:]

In [209]:
p.sort_values()[-5:]

293      645.600649
1914     681.339159
1157    1009.253268
388     1489.838235
389     8898.897059
dtype: float64

In [210]:
m2010.loc[293]
m2010.loc[1914]
m2010.loc[1157]
m2010.loc[388]
m2010.loc[389]

state            District of Columbia
state/region                       DC
ages                            total
year                             2010
population                     605125
area (sq. mi)                      68
Name: 389, dtype: object

找出人口密度最低的五个州

In [211]:
p.sort_values()[0:5]

90      0.286251
91      1.087509
2404    1.383702
964     1.518518
1348    2.124052
dtype: float64

In [214]:
m2010.loc[90]
# m2010.loc[91]
# m2010.loc[2404]
# m2010.loc[964]
# m2010.loc[1348]

state             Alaska
state/region          AK
ages             under18
year                2010
population        187902
area (sq. mi)     656425
Name: 90, dtype: object

要点总结：
- 统一用loc()索引
- 善于使用.isnull().any()找到存在NaN的列
- 善于使用.unique()确定该列中哪些key是我们需要的
- 一般使用外合并、左合并，目的只有一个：宁愿该列是NaN也不要丢弃其他列的信息

## 回顾：Series/DataFrame运算与ndarray运算的区别

- Series与DataFrame没有广播，如果对应index没有值，则记为NaN；或者使用add的fill_value来补缺失值
- ndarray有广播，通过重复已有值来计算